<a href="https://colab.research.google.com/github/iam-Dylan/automated-essay-scoring/blob/dagngyen/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>
    h1 {
        padding: 8px 8px;
        background-image: linear-gradient(135deg, #c9f3ff, rgb(131, 218, 255));
        font-weight: 700;
        position: static;
        text-align: center;
        color: #006098;
        font-size: 36px;
    }
    h2 {
        font-weight: 700;
        text-align: center;
        font-style: italic;
        font-size: 24px;
    }
</style>

<div><h1>XÂY DỰNG MODEL</h1></div>
<div><h2>RAPIDS cuML SVR model</h2></div>

# Khai báo thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



# Đọc dữ liệu

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
train = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/train.csv')
train.sample(5)

,essay_id,full_text,score
7088,6a3ad47,Driverless cars next big thing ? Sergey Brian ...,2
1773,1b5f4fa,Driverless cars could be the way of the future...,4
1840,1c4c81e,I believe the use of the technology to read st...,4
1979,1e2c5f1,Driveless cars are coming. Through the passage...,3
1081,1100c54,"Dear senator,\n\nMany say that the Electoral C...",2


In [4]:
test = pd.read_csv('/gdrive/MyDrive/Colab Notebooks/data/test.csv')
test

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


# Xử lý dữ liệu thông qua trích xuất embeddings từ các mô hình Hugging Face

## Cài đặt thư viện cần thiết

In [5]:
# !pip install transformers torch cudf cuml

In [6]:
# !pip install transformers torch scikit-learn pandas

In [7]:
from transformers import AutoTokenizer, AutoModel
import torch

models = [
    "bert-base-uncased",
    "roberta-base",
    "xlnet-base-cased",
    "gpt2",
    "distilbert-base-uncased",
    "albert-base-v2"
]

tokenizers = [AutoTokenizer.from_pretrained(model) for model in models]
models = [AutoModel.from_pretrained(model) for model in models]

# Thêm token padding cho các tokenizer không có nó
for tokenizer in tokenizers:
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probab

In [ ]:
essays = train['full_text'].to_numpy()  # Danh sách các bài luận cần xử lý
tokenized_essays = []

for tokenizer in tokenizers:
    tokenized_essay = [tokenizer(essay, padding='max_length', truncation=True, return_tensors='pt', max_length=1024) for essay in essays]
    tokenized_essays.append(tokenized_essay)


In [ ]:
embeddings = []

for model, tokenized in zip(models, tokenized_essays):
    model_embeddings = []
    for essay in tokenized:
        with torch.no_grad():
            outputs = model(**essay)
            # Sử dụng output ẩn cuối cùng
            model_embeddings.append(outputs.last_hidden_state[:, 0, :].numpy())
    embeddings.append(np.array(model_embeddings))

# Nối tất cả các embeddings lại với nhau
combined_embeddings = np.concatenate(embeddings, axis=-1).reshape(len(essays), -1)